# Weather Predictions blender

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from glob import glob
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

import shutil

# Project
project_common_path = os.path.dirname('.')
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)

In [3]:
os.environ['THEANO_FLAGS'] = 'device=cpu'

import numpy as np
import pandas as pd

from data_utils import get_id_type_list_for_class, GENERATED_DATA, OUTPUT_PATH
from test_utils import create_submission

No test data found at D:\vfomin\MyExamples\python\planet_amazon_rainforest\common\..\input\test 
No additional test data found at D:\vfomin\MyExamples\python\planet_amazon_rainforest\common\..\input\test 


Using TensorFlow backend.


### Load predictions on trainval dataset

In [4]:
from data_utils import unique_tags, get_label

target_tags = ['target_' + t for t in unique_tags]
val_predictions_resnet_filepath = os.path.join(GENERATED_DATA, "val_predictions_ResNet50_2017-07-18-14-50.csv")
val_predictions_squeezenet_filepath = os.path.join(GENERATED_DATA, "val_predictions_SqueezeNet21_2017-07-13-22-23.csv")

val_predictions_vgg19_filepath = os.path.join(GENERATED_DATA, "val_predictions_deep_model_vgg19_train_LB092919_prob.csv")
val_predictions_weirdcnn_filepath = os.path.join(GENERATED_DATA, "val_predictions_deep_model_train_LB092655_prob.csv")

def get_val_predictions_df(val_predictions_filepath, search_prefix="val_predictions_*.csv"):
    if not os.path.exists(val_predictions_filepath):
        val_predictions_csv = glob(os.path.join(OUTPUT_PATH, search_prefix))
        df = pd.read_csv(val_predictions_csv[0]).dropna()
        for filepath in val_predictions_csv[1:]:
            df = pd.concat([df, pd.read_csv(filepath).dropna()])
        df.reset_index(inplace=True)   
        df.drop('index', axis=1, inplace=True)
        df['image_id'] = df['image_name'].apply(lambda x: int(x[len('train_'):]))    
        for t in target_tags:
            df[t] = ''
        def fill_target_tags(row):
            image_id = row[0]
            labels = get_label(image_id, "Train_jpg")
            row[1:] = labels    
            return row
        cols = ['image_id', ] + target_tags
        df[cols] = df[cols].apply(fill_target_tags, axis=1)

        df.to_csv(val_predictions_filepath, index=False)
        val_predictions_df = df
        df = None    
    else:
        val_predictions_df = pd.read_csv(val_predictions_filepath)
    return val_predictions_df


# val_predictions_resnet_df = get_val_predictions_df(val_predictions_resnet_filepath, "val_predictions_ResNet50*_2017-07-18-14-50.csv")
# val_predictions_squeezenet_df = get_val_predictions_df(val_predictions_squeezenet_filepath, "val_predictions_SqueezeNet21*_2017-07-18-14-50.csv")
val_predictions_vgg19_df = get_val_predictions_df(val_predictions_vgg19_filepath, "vgg19/deep_model_vgg19_train_LB092919_prob.csv")
val_predictions_weirdcnn_df = get_val_predictions_df(val_predictions_weirdcnn_filepath, "custom_weird_model/deep_model_train_LB092655_prob.csv")


In [7]:
print(len(val_predictions_resnet_df))
val_predictions_resnet_df.head()

40448


,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,...,target_conventional_mine,target_cultivation,target_habitation,target_haze,target_partly_cloudy,target_primary,target_road,target_selective_logging,target_slash_burn,target_water
0,train_18872,0.762381,0.007814,0.023866,0.002480,0.001474,0.983301,0.000370,0.002931,0.932843,...,0,1,1,0,0,1,0,0,1,1
1,train_10562,0.106635,0.000834,0.002179,0.001490,0.001146,0.000112,0.003351,0.001014,0.020308,...,0,0,0,0,1,1,0,0,0,0
2,train_28763,0.196588,0.004786,0.009777,0.002631,0.001869,0.019417,0.105915,0.004077,0.084854,...,0,1,1,0,1,1,0,0,0,0
3,train_31735,0.084816,0.001532,0.013537,0.010144,0.014883,0.633817,0.002344,0.002039,0.045785,...,0,0,0,0,0,1,0,0,0,0
4,train_320,0.106061,0.001101,0.004905,0.004541,0.001737,0.000195,0.000810,0.001513,0.102018,...,0,0,0,0,1,1,0,0,0,0


In [8]:
print(len(val_predictions_squeezenet_df))
val_predictions_squeezenet_df.head()

40320


,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,...,target_conventional_mine,target_cultivation,target_habitation,target_haze,target_partly_cloudy,target_primary,target_road,target_selective_logging,target_slash_burn,target_water
0,train_15438,0.986593,1.800301e-07,0.016957,3.479813e-08,2.728809e-07,0.986830,1.494415e-08,2.339067e-04,0.128825,...,0,1,0,1,0,1,1,0,0,0
1,train_12863,0.024612,7.483472e-08,0.000639,8.198895e-03,1.618286e-03,0.997336,1.701083e-08,9.840907e-09,0.007981,...,0,0,0,0,0,1,0,0,0,0
2,train_10519,0.721716,5.289071e-05,0.014284,1.069636e-04,1.935463e-03,0.000348,2.059214e-05,3.164162e-05,0.509124,...,0,0,0,0,1,1,0,0,0,0
3,train_17178,0.002469,9.790796e-11,0.000020,1.204151e-03,8.132902e-05,0.998747,8.051727e-10,1.947120e-11,0.000470,...,0,0,0,0,0,1,0,0,0,0
4,train_5177,0.066280,3.975656e-07,0.001354,5.240596e-04,2.015755e-04,0.348959,4.480021e-04,5.901127e-07,0.016081,...,0,0,0,0,0,1,0,0,0,0


In [14]:
len(val_predictions_resnet_df['image_name'].unique()), len(val_predictions_squeezenet_df['image_name'].unique()), 

(27158, 31357)

In [79]:
from metrics import score

def get_optimal_thresholds(y_true, y_preds):
    best_thresholds = [0.0]*len(unique_tags)    
    best_score = 0
    thrs = np.arange(0.0, 1.01, 0.01)    
    for i, tag in enumerate(unique_tags):
        print("%s : best_score=" % tag, end="")
        thresholds = list(best_thresholds)
        for thr in thrs:            
            thresholds[i] = thr
            s = score(y_true, y_preds > thresholds)
            if s > best_score:
                best_score = s
                best_thresholds[i] = thr
        print("%f, best_threshold=%f" % (best_score, best_thresholds[i]))
    return best_thresholds, best_score

## Train xgboost trees to make better predict weather

### Train to predict weather classes : 

In [33]:
weather_tags = ['clear', 'cloudy', 'haze', 'partly_cloudy']
weather_target_tags = ['target_%s' % t for t in weather_tags]
weather_labels = np.where(np.isin(unique_tags, weather_tags))[0]

In [34]:
weather_labels, [(i, t) for i, t in enumerate(unique_tags)], weather_target_tags

(array([ 5,  6, 10, 11], dtype=int64),
 [(0, 'agriculture'),
  (1, 'artisinal_mine'),
  (2, 'bare_ground'),
  (3, 'blooming'),
  (4, 'blow_down'),
  (5, 'clear'),
  (6, 'cloudy'),
  (7, 'conventional_mine'),
  (8, 'cultivation'),
  (9, 'habitation'),
  (10, 'haze'),
  (11, 'partly_cloudy'),
  (12, 'primary'),
  (13, 'road'),
  (14, 'selective_logging'),
  (15, 'slash_burn'),
  (16, 'water')],
 ['target_clear', 'target_cloudy', 'target_haze', 'target_partly_cloudy'])

In [35]:
val_predictions_df = val_predictions_vgg19_df

In [36]:
val_predictions_df.head()

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,...,target_conventional_mine,target_cultivation,target_habitation,target_haze,target_partly_cloudy,target_primary,target_road,target_selective_logging,target_slash_burn,target_water
0,0.003752,1.428458e-09,0.000088,0.000230,0.000018,0.581344,2.108085e-05,2.150840e-09,0.001460,0.000500,...,0,0,0,1,0,1,0,0,0,0
1,0.799365,1.561381e-04,0.009740,0.000171,0.000109,0.998879,1.573375e-06,1.418633e-04,0.128822,0.038553,...,0,0,0,0,0,1,0,0,0,1
2,0.001674,1.083242e-09,0.000018,0.000292,0.000128,0.999015,1.269098e-06,8.769292e-11,0.000902,0.000416,...,0,0,0,0,0,1,0,0,0,0
3,0.005073,3.921103e-08,0.000086,0.006642,0.000504,0.998185,2.577778e-06,3.010564e-09,0.002385,0.002778,...,0,0,0,0,0,1,0,0,0,0
4,0.879332,1.331866e-03,0.017324,0.001006,0.000401,0.998223,4.311219e-07,3.768842e-04,0.343372,0.786686,...,0,0,1,0,0,1,1,0,0,0


In [10]:
import xgboost as xgb

d:\vfomin\myexamples\venv3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### A simple try of xgb

In [220]:
from sklearn.model_selection import StratifiedKFold

In [159]:
n_folds = 3

In [160]:
m = val_predictions_df[weather_target_tags].sum(axis=1) == 1
m2 = ~m

In [227]:

trainval_x = val_predictions_df[m][unique_tags].values
trainval_y = val_predictions_df[m][weather_target_tags].values

d = {
    (1, 0, 0, 0) : 0,
    (0, 1, 0, 0) : 1,    
    (0, 0, 1, 0) : 2,   
    (0, 0, 0, 1) : 3,    
}

def vector_to_index(trainval_y):
    
    output = np.zeros(len(trainval_y), dtype=np.uint8)
    for i, v in enumerate(trainval_y):        
        output[i] = d[tuple(v.tolist())]
    return output

trainval_y_ = vector_to_index(trainval_y)
    
skf = StratifiedKFold(n_splits=n_folds)
for train_index, test_index in skf.split(trainval_x, trainval_y_):
    train_x, val_x = trainval_x[train_index], trainval_x[test_index]
    train_y, val_y = trainval_y_[train_index], trainval_y_[test_index]
    
    print(train_x.shape, train_y.shape)
    print(train_x[:5, :], train_y[:5])
    print(val_x[:5, :], val_y[:5])
    break

(32380, 17) (32380,)
[[  6.04965031e-01   2.15695635e-03   7.51044676e-02   1.01612241e-03
    4.62560332e-04   1.75208211e-01   2.06295023e-04   7.55776477e-04
    5.43331623e-01   1.54359359e-02   8.81842554e-01   1.84541978e-02
    9.98271704e-01   3.75210531e-02   4.39244695e-03   4.38418332e-03
    2.68821627e-01]
 [  1.63640738e-01   8.02634804e-06   3.85102024e-03   1.35511495e-04
    1.37903524e-04   8.12684834e-01   2.27927212e-02   3.22563865e-05
    6.34065364e-03   7.66972499e-03   1.26849264e-01   1.25037991e-02
    9.31428313e-01   6.91066682e-01   3.30558891e-04   6.47022243e-05
    7.80763984e-01]
 [  4.39591426e-03   1.74956183e-09   7.79319234e-05   3.57810524e-04
    2.33775190e-05   6.20837331e-01   1.54425688e-05   2.05203743e-09
    1.48288393e-03   6.66052103e-04   4.02874887e-01   1.91877212e-03
    9.99889016e-01   3.08704283e-03   2.04650041e-05   5.34883384e-07
    4.17424552e-03]
 [  9.81193781e-02   5.00835449e-06   1.99962687e-03   2.17977031e-05
    2.905

In [243]:
params = {
    "objective": "multi:softprob",
    "booster": "gbtree",
    "eval_metric": "mlogloss",
    "learning_rate": 0.007,
    "tree_method": 'exact',
    "n_estimators": 150,
    "max_depth": 4,
    "subsample": 0.75,
    "colsample_bytree": 0.75,
    "silent": False,    
    "num_class": len(weather_target_tags)
}
num_boost_round = 2500
early_stopping_rounds = 100

In [231]:
dtrain = xgb.DMatrix(train_x, label=train_y)
dval = xgb.DMatrix(val_x, label=val_y)

In [232]:
watchlist = ((dtrain, 'train'), (dval, 'eval'))

In [233]:
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)

[0]	train-mlogloss:1.37389	eval-mlogloss:1.37389
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 100 rounds.
[1]	train-mlogloss:1.36161	eval-mlogloss:1.36162
[2]	train-mlogloss:1.34963	eval-mlogloss:1.34967
[3]	train-mlogloss:1.33782	eval-mlogloss:1.33786
[4]	train-mlogloss:1.32672	eval-mlogloss:1.32678
[5]	train-mlogloss:1.31574	eval-mlogloss:1.31581
[6]	train-mlogloss:1.30431	eval-mlogloss:1.30439
[7]	train-mlogloss:1.2931	eval-mlogloss:1.29319
[8]	train-mlogloss:1.28227	eval-mlogloss:1.28238
[9]	train-mlogloss:1.2713	eval-mlogloss:1.27142
[10]	train-mlogloss:1.26049	eval-mlogloss:1.26062
[11]	train-mlogloss:1.25066	eval-mlogloss:1.25079
[12]	train-mlogloss:1.24018	eval-mlogloss:1.24032
[13]	train-mlogloss:1.23111	eval-mlogloss:1.23128
[14]	train-mlogloss:1.22132	eval-mlogloss:1.22151
[15]	train-mlogloss:1.21117	eval-mlogloss:1.21136
[16]	train-mlogloss:1.20123	eval-mlogloss:1.20142
[17]	train

Compare performances :

Train

In [234]:
y_true = val_predictions_df[m][target_tags].values[train_index]
y_preds_before = val_predictions_df[m][unique_tags].values[train_index]
# get_optimal_thresholds(y_true, y_preds_before)

```
agriculture : best_score=0.492248, best_threshold=0.140000
artisinal_mine : best_score=0.510153, best_threshold=0.180000
bare_ground : best_score=0.527707, best_threshold=0.080000
blooming : best_score=0.547661, best_threshold=0.110000
blow_down : best_score=0.570279, best_threshold=0.160000
clear : best_score=0.575409, best_threshold=0.150000
cloudy : best_score=0.598786, best_threshold=0.090000
conventional_mine : best_score=0.626382, best_threshold=0.090000
cultivation : best_score=0.648190, best_threshold=0.180000
habitation : best_score=0.676076, best_threshold=0.170000
haze : best_score=0.707198, best_threshold=0.100000
partly_cloudy : best_score=0.738221, best_threshold=0.190000
primary : best_score=0.739737, best_threshold=0.210000
road : best_score=0.773801, best_threshold=0.180000
selective_logging : best_score=0.823794, best_threshold=0.100000
slash_burn : best_score=0.882304, best_threshold=0.140000
water : best_score=0.935892, best_threshold=0.170000

([0.14000000000000001,   0.17999999999999999,  0.080000000000000002,  0.11,  0.16,  0.14999999999999999,  0.089999999999999997, 0.089999999999999997,  0.17999999999999999,  0.17000000000000001,  0.10000000000000001,  0.19,  0.20999999999999999, 0.17999999999999999,  0.10000000000000001,  0.14000000000000001,  0.17000000000000001], 0.93589220897257264)
```

In [235]:
def check_predictions(y_true, y_preds_before, y_preds_after):
    
    thrs = np.array([0.14000000000000001,   0.17999999999999999,  0.080000000000000002,  0.11,  0.16,  0.14999999999999999,  0.089999999999999997, 0.089999999999999997,  0.17999999999999999,  0.17000000000000001,  0.10000000000000001,  0.19,  0.20999999999999999, 0.17999999999999999,  0.10000000000000001,  0.14000000000000001,  0.17000000000000001])
    print("No boosting. Mean train error (with best thresholds): ", np.mean(np.abs(y_true - (y_preds_before > thrs).astype(np.uint8))))
    print("No boosting. Mean weather only train error (with best thresholds): ", np.mean(np.abs(y_true[:, weather_labels] - (y_preds_before[:, weather_labels] > thrs[weather_labels]).astype(np.uint8))))
    
    print("With boosting. Mean weather only train error: ", np.mean(np.abs(y_true[:, weather_labels] - (y_preds_after[:, weather_labels] > 0.5).astype(np.uint8))))
    print("With boosting. Mean train error (with threshold = 0.5): ", np.mean(np.abs(y_true - (y_preds_after > 0.5).astype(np.uint8))))
    
    print(get_optimal_thresholds(y_true, y_preds_after))
    

In [236]:
y_preds_after = y_preds_before.copy()
y_preds_weather_after = gbm.predict(xgb.DMatrix(train_x), ntree_limit=gbm.best_iteration+1)
y_preds_after[:, weather_labels] = y_preds_weather_after

check_predictions(y_true, y_preds_before, y_preds_after)

No boosting. Mean train error (with best thresholds):  0.045698143371
No boosting. Mean weather only train error (with best thresholds):  0.045460160593
With boosting. Mean weather only train error:  0.0245521927116
With boosting. Mean train error (with threshold = 0.5):  0.0285524833775
agriculture : best_score=0.492220, best_threshold=0.140000
artisinal_mine : best_score=0.510126, best_threshold=0.180000
bare_ground : best_score=0.527628, best_threshold=0.110000
blooming : best_score=0.547611, best_threshold=0.110000
blow_down : best_score=0.570212, best_threshold=0.110000
clear : best_score=0.575589, best_threshold=0.100000
cloudy : best_score=0.599290, best_threshold=0.130000
conventional_mine : best_score=0.626900, best_threshold=0.090000
cultivation : best_score=0.648638, best_threshold=0.180000
habitation : best_score=0.676545, best_threshold=0.180000
haze : best_score=0.708313, best_threshold=0.160000
partly_cloudy : best_score=0.739784, best_threshold=0.210000
primary : best_s

Test

In [237]:
y_true = val_predictions_df[m][target_tags].values[test_index]
y_preds_before = val_predictions_df[m][unique_tags].values[test_index]

In [170]:
# get_optimal_thresholds(y_true, y_preds_before)

```
agriculture : best_score=0.491723, best_threshold=0.140000
artisinal_mine : best_score=0.509622, best_threshold=0.240000
bare_ground : best_score=0.527223, best_threshold=0.130000
blooming : best_score=0.547259, best_threshold=0.090000
blow_down : best_score=0.569750, best_threshold=0.060000
clear : best_score=0.575042, best_threshold=0.130000
cloudy : best_score=0.598413, best_threshold=0.070000
conventional_mine : best_score=0.626027, best_threshold=0.270000
cultivation : best_score=0.647533, best_threshold=0.210000
habitation : best_score=0.675459, best_threshold=0.140000
haze : best_score=0.706729, best_threshold=0.120000
partly_cloudy : best_score=0.737847, best_threshold=0.190000
primary : best_score=0.739535, best_threshold=0.170000
road : best_score=0.773389, best_threshold=0.220000
selective_logging : best_score=0.823530, best_threshold=0.170000
slash_burn : best_score=0.882076, best_threshold=0.200000
water : best_score=0.935354, best_threshold=0.210000

([0.14000000000000001,   0.23999999999999999,  0.13,  0.089999999999999997,  0.059999999999999998,  0.13,  0.070000000000000007, 0.27000000000000002,  0.20999999999999999,  0.14000000000000001,  0.12,  0.19,  0.17000000000000001,  0.22, 0.17000000000000001, 0.20000000000000001,  0.20999999999999999], 0.93535448764300033)
```

In [238]:
y_preds_after = y_preds_before.copy()
y_preds_weather_after = gbm.predict(xgb.DMatrix(val_x), ntree_limit=gbm.best_iteration+1)
y_preds_after[:, weather_labels] = y_preds_weather_after

check_predictions(y_true, y_preds_before, y_preds_after)

No boosting. Mean train error (with best thresholds):  0.0456975578574
No boosting. Mean weather only train error (with best thresholds):  0.0441467028896
With boosting. Mean weather only train error:  0.0286490491479
With boosting. Mean train error (with threshold = 0.5):  0.0293972367905
agriculture : best_score=0.491539, best_threshold=0.130000
artisinal_mine : best_score=0.509423, best_threshold=0.220000
bare_ground : best_score=0.527188, best_threshold=0.130000
blooming : best_score=0.547149, best_threshold=0.070000
blow_down : best_score=0.569624, best_threshold=0.060000
clear : best_score=0.574860, best_threshold=0.130000
cloudy : best_score=0.598185, best_threshold=0.030000
conventional_mine : best_score=0.625894, best_threshold=0.360000
cultivation : best_score=0.647580, best_threshold=0.210000
habitation : best_score=0.675544, best_threshold=0.140000
haze : best_score=0.706919, best_threshold=0.110000
partly_cloudy : best_score=0.737968, best_threshold=0.090000
primary : best

Run CV with a random search of optimal parameters

In [240]:
def generate_params(iter_num, seed=None):
    if iter_num > 0:
        if seed is not None:
            np.random.seed(seed=seed)
        for z in range(iter_num):
            eta = np.random.uniform(0.05, 0.001)
            max_depth = np.random.randint(2, 6)
            subsample = np.random.uniform(0.5, 0.95)
            colsample_bytree = np.random.uniform(0.5, 0.95)
            yield eta, max_depth, subsample, colsample_bytree
    else:
        eta = 0.05
        max_depth = 3
        subsample = 0.8204967474962096
        colsample_bytree = 0.7089159774987868
        yield eta, max_depth, subsample, colsample_bytree

In [241]:
m = val_predictions_df[weather_target_tags].sum(axis=1) == 1
trainval_x = val_predictions_df[m][unique_tags].values
trainval_y = val_predictions_df[m][weather_target_tags].values

d = {
    (1, 0, 0, 0) : 0,
    (0, 1, 0, 0) : 1,    
    (0, 0, 1, 0) : 2,   
    (0, 0, 0, 1) : 3,    
}

def vector_to_index(trainval_y):
    
    output = np.zeros(len(trainval_y), dtype=np.uint8)
    for i, v in enumerate(trainval_y):        
        output[i] = d[tuple(v.tolist())]
    return output

trainval_y_ = vector_to_index(trainval_y)


In [242]:

eval_metric = 'mlogloss'

seed = 2017
n_folds = 5
dtrainval = xgb.DMatrix(trainval_x, label=trainval_y_, feature_names=unique_tags)

best_params = {
    'test-%s-mean' % eval_metric: 1e10,
    'params': None,
}
    
iter_num = 10
for (eta, max_depth, subsample, colsample_bytree) in list(generate_params(iter_num, seed=seed)):
    print('XGBoost params. ETA: {}, MAX_DEPTH: {}, SUBSAMPLE: {}, COLSAMPLE_BY_TREE: {}'.format(eta, max_depth,subsample,colsample_bytree))
    params = {
        "objective": "multi:softprob",
        "booster": "gbtree",
        "eval_metric": eval_metric,
        "eta": eta,
        "tree_method": 'exact',
        "max_depth": max_depth,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "silent": 1,
        "seed": seed, 
        "num_class": len(weather_target_tags)
    }
    num_boost_round = 2500
    early_stopping_rounds = 100

    cvresult = xgb.cv(params, dtrain=dtrainval,
                       seed=params['seed'], stratified=True,
                       num_boost_round=num_boost_round, 
                       early_stopping_rounds=early_stopping_rounds, nfold=n_folds, verbose_eval=False)

    min_test_logloss_mean = cvresult['test-%s-mean' % params['eval_metric']].min()
    if best_params['test-%s-mean' % params['eval_metric']] > min_test_logloss_mean:
        best_params['test-%s-mean' % params['eval_metric']] = min_test_logloss_mean
        best_params['params'] = params
        print("Best cv result: ", cvresult.loc[cvresult.index[-1], :])
        print("Best params: ", params)
            
        

XGBoost params. ETA: 0.04897294895510025, MAX_DEPTH: 4, SUBSAMPLE: 0.8241212099907093, COLSAMPLE_BY_TREE: 0.8849532533753195
Best cv result:  test-mlogloss-mean     0.144753
test-mlogloss-std      0.004451
train-mlogloss-mean    0.120154
train-mlogloss-std     0.000932
Name: 184, dtype: float64
Best params:  {'objective': 'multi:softprob', 'booster': 'gbtree', 'eval_metric': 'mlogloss', 'eta': 0.04897294895510025, 'tree_method': 'exact', 'max_depth': 4, 'subsample': 0.8241212099907093, 'colsample_bytree': 0.8849532533753195, 'silent': 1, 'seed': 2017, 'num_class': 4}
XGBoost params. ETA: 0.01222624046647642, MAX_DEPTH: 5, SUBSAMPLE: 0.7922976846990782, COLSAMPLE_BY_TREE: 0.5633019770976554
XGBoost params. ETA: 0.03865187746016863, MAX_DEPTH: 2, SUBSAMPLE: 0.9309433282690102, COLSAMPLE_BY_TREE: 0.9139775309770741
Best cv result:  test-mlogloss-mean     0.143572
test-mlogloss-std      0.004617
train-mlogloss-mean    0.135080
train-mlogloss-std     0.001078
Name: 303, dtype: float64
Best 

KeyboardInterrupt: 

In [246]:
best_params['params'] = params

In [ ]:
best_params

Train xgboost model on all data

In [248]:
sss = StratifiedShuffleSplit()

for train_index, test_index in sss.split(trainval_x, trainval_y_):
    train_x, val_x = trainval_x[train_index], trainval_x[test_index]
    train_y, val_y = trainval_y_[train_index], trainval_y_[test_index]
    break
    
dtrain = xgb.DMatrix(train_x, label=train_y)
dval = xgb.DMatrix(val_x, label=val_y)
watchlist = [(dtrain, 'train'), (dval, 'eval')]
params = best_params['params']
num_boost_round = 2500
early_stopping_rounds = 100

gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)                
print("Best score: ", gbm.best_score)
best_params['gbm'] = gbm

y_true = val_predictions_df[m][target_tags].values[test_index]
y_preds_before = val_predictions_df[m][unique_tags].values[test_index]
y_preds_after = y_preds_before.copy()
y_preds_weather_after = gbm.predict(xgb.DMatrix(val_x), ntree_limit=gbm.best_iteration+1)
y_preds_after[:, weather_labels] = y_preds_weather_after

print("With boosting. Mean weather only train error: ", np.mean(np.abs(y_true[:, weather_labels] - (y_preds_after[:, weather_labels] > 0.5).astype(np.uint8))))
print("With boosting. Mean train error (with threshold = 0.5): ", np.mean(np.abs(y_true - (y_preds_after > 0.5).astype(np.uint8))))
    
best_thresholds, best_score = get_optimal_thresholds(y_true, y_preds_after)


[0]	train-mlogloss:1.37398	eval-mlogloss:1.37401
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 100 rounds.
[1]	train-mlogloss:1.36254	eval-mlogloss:1.36264
[2]	train-mlogloss:1.35043	eval-mlogloss:1.35055
[3]	train-mlogloss:1.33875	eval-mlogloss:1.33891
[4]	train-mlogloss:1.32734	eval-mlogloss:1.32755
[5]	train-mlogloss:1.31629	eval-mlogloss:1.31656
[6]	train-mlogloss:1.30485	eval-mlogloss:1.30516
[7]	train-mlogloss:1.29357	eval-mlogloss:1.2939
[8]	train-mlogloss:1.28249	eval-mlogloss:1.28283
[9]	train-mlogloss:1.27177	eval-mlogloss:1.27215
[10]	train-mlogloss:1.26106	eval-mlogloss:1.26147
[11]	train-mlogloss:1.25039	eval-mlogloss:1.25082
[12]	train-mlogloss:1.24008	eval-mlogloss:1.24055
[13]	train-mlogloss:1.22968	eval-mlogloss:1.23018
[14]	train-mlogloss:1.21955	eval-mlogloss:1.22007
[15]	train-mlogloss:1.20974	eval-mlogloss:1.2103
[16]	train-mlogloss:1.19996	eval-mlogloss:1.20056
[17]	train

NameError: name 'best_threholds' is not defined

In [275]:
print(best_thresholds, best_score)
best_params['best_thresholds'] = best_thresholds
best_params['best_score'] = best_score

[0.14000000000000001, 0.26000000000000001, 0.16, 0.080000000000000002, 0.10000000000000001, 0.17000000000000001, 0.12, 0.35999999999999999, 0.23999999999999999, 0.22, 0.17999999999999999, 0.10000000000000001, 0.19, 0.23999999999999999, 0.14999999999999999, 0.17000000000000001, 0.16] 0.939732739146


In [250]:
import pickle 
now = datetime.now()

xgb_best_params_filepath = os.path.join(GENERATED_DATA, 'xgb_best_params_%s.pkl' % str(now.strftime("%Y-%m-%d-%H-%M")))

with open(xgb_best_params_filepath, 'wb') as handle:
    pickle.dump(best_params, handle, protocol=pickle.HIGHEST_PROTOCOL)    

Boost test predictions

In [259]:
# predictions_csv = glob(os.path.join(OUTPUT_PATH, "vgg", "deep_model_vgg19_test_LB092919_prob.csv"))
predictions_csv = [os.path.join(OUTPUT_PATH, "vgg19", "deep_model_vgg19_test_LB092919_prob.csv"), ]

prediction_df = pd.read_csv(predictions_csv[0]).dropna()
for filepath in predictions_csv[1:]:
    prediction_df = pd.concat([prediction_df, pd.read_csv(filepath).dropna()])
prediction_df.reset_index(inplace=True)   
prediction_df.drop('index', axis=1, inplace=True)

In [260]:
prediction_df.head()

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water,image_name
0,0.006536,3.512421e-08,0.000097,0.016582,0.000294,0.989730,0.000015,6.567299e-09,0.003071,0.003682,0.008398,0.001877,0.999895,0.001971,0.001498,0.000012,0.003857,test_0
1,0.009448,1.365362e-07,0.000259,0.014264,0.003731,0.999242,0.000001,1.036118e-08,0.004254,0.002162,0.000075,0.001190,0.999959,0.001835,0.004290,0.000053,0.003081,test_1
2,0.014498,9.787799e-07,0.000161,0.000073,0.000157,0.001720,0.000583,3.955855e-07,0.002629,0.000157,0.000525,0.996830,0.999648,0.005254,0.000077,0.000038,0.041479,test_2
3,0.233005,2.386333e-05,0.001563,0.006978,0.001638,0.807728,0.000011,3.210328e-05,0.099639,0.001558,0.006070,0.194216,0.999919,0.024983,0.003209,0.000523,0.042516,test_3
4,0.005320,1.746132e-07,0.000232,0.000026,0.000071,0.001468,0.333464,1.131022e-07,0.005054,0.000125,0.005073,0.705048,0.683056,0.003038,0.000006,0.000012,0.007390,test_4


In [261]:
test_x = prediction_df[unique_tags].values
y_preds_after = test_x.copy()

dtest = xgb.DMatrix(test_x)
gbm = best_params['gbm']
y_preds_weather_after = gbm.predict(dtest, ntree_limit=gbm.best_iteration+1)        
y_preds_after[:, weather_labels] = y_preds_weather_after


In [269]:
df = prediction_df.copy()
df[unique_tags] = y_preds_after

In [270]:
df.head()

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water,image_name
0,0.006536,3.512421e-08,0.000097,0.016582,0.000294,0.992109,0.000132,6.567299e-09,0.003071,0.003682,0.007370,0.000389,0.999895,0.001971,0.001498,0.000012,0.003857,test_0
1,0.009448,1.365362e-07,0.000259,0.014264,0.003731,0.999713,0.000027,1.036118e-08,0.004254,0.002162,0.000078,0.000182,0.999959,0.001835,0.004290,0.000053,0.003081,test_1
2,0.014498,9.787799e-07,0.000161,0.000073,0.000157,0.000451,0.000103,3.955855e-07,0.002629,0.000157,0.000202,0.999244,0.999648,0.005254,0.000077,0.000038,0.041479,test_2
3,0.233005,2.386333e-05,0.001563,0.006978,0.001638,0.860062,0.000425,3.210328e-05,0.099639,0.001558,0.010285,0.129228,0.999919,0.024983,0.003209,0.000523,0.042516,test_3
4,0.005320,1.746132e-07,0.000232,0.000026,0.000071,0.003753,0.271258,1.131022e-07,0.005054,0.000125,0.008130,0.716859,0.683056,0.003038,0.000006,0.000012,0.007390,test_4


In [271]:
def compute_mean(df):
    gb = df.groupby('image_name')
    df2 = gb.agg(np.mean).reset_index()
    return df2

In [272]:
mean_df = compute_mean(df)

In [279]:
thresholds = {}

for i, tag in enumerate(unique_tags):
    thresholds[tag] = best_params['best_thresholds'][i]

thresholds

{'agriculture': 0.14000000000000001,
 'artisinal_mine': 0.26000000000000001,
 'bare_ground': 0.16,
 'blooming': 0.080000000000000002,
 'blow_down': 0.10000000000000001,
 'clear': 0.17000000000000001,
 'cloudy': 0.12,
 'conventional_mine': 0.35999999999999999,
 'cultivation': 0.23999999999999999,
 'habitation': 0.22,
 'haze': 0.17999999999999999,
 'partly_cloudy': 0.10000000000000001,
 'primary': 0.19,
 'road': 0.23999999999999999,
 'selective_logging': 0.14999999999999999,
 'slash_burn': 0.17000000000000001,
 'water': 0.16}

In [281]:
# create_submission(mean_df, info="vgg19_boosted_weather", thresholds=best_params['best_thresholds'])
create_submission(df, info="vgg19_boosted_weather", thresholds=thresholds)